## Project 5: Contagion risk mapper
**Goal**:
Simulate infection spread using symptom onset and contact logs — and flag crew members who had close contact with a symptomatic individual in the days leading up to their symptom onset.

### 1. Case identification

In [1]:
import pandas as pd

In [2]:
contagion_log = pd.read_csv('./data/contagion_log.csv')

In [3]:
contagion_log.head()

,date,crew_member,contact_with,symptoms
0,2025-03-20,Alice Carter,Ben O'Hara,No
1,2025-03-20,Ben O'Hara,Alice Carter,No
2,2025-03-20,Jonas Müller,Nina Patel,No
3,2025-03-20,Nina Patel,Leo Zhang,No
4,2025-03-20,Leo Zhang,Nina Patel,No


In [4]:
symptomatic = contagion_log[contagion_log['symptoms'] == 'Yes']

In [5]:
symptomatic

,date,crew_member,contact_with,symptoms
45,2025-03-27,Nina Patel,Leo Zhang,Yes
47,2025-03-27,Emma López,Alice Carter,Yes


On 2025-03-27, both:
- Nina Patel
- Emma López

became symptomatic (symptoms == "Yes"). This can be regarded as Day 0 for potential transmission.

### 2. Backward tracing - identifying suspected Source contacts
What happened before Nina Patel and Emma López became symptomatic?
Did one infect the other — or could other crew members have served as potential pre-symptomatic sources or amplifiers within the transmission chain, despite not exhibiting symptoms themselves?

Below we look at Nina Patel and Emma López's contact events in the 3 days before symptom onset, to identify individuals who may have contributed to the spread of infection through close proximity or repeated interactions.

In [12]:
def contact_tracing(names):
    for name in names:
        filter_name = contagion_log[contagion_log['crew_member']== name]
        filter_name = filter_name[(filter_name['date'] >= '2025-03-24') & (filter_name['date'] <='2025-03-26')]
        print(f"\n{filter_name}")

In [13]:
contact_tracing(['Nina Patel','Emma López'])


          date crew_member  contact_with symptoms
27  2025-03-24  Nina Patel  Alice Carter       No
33  2025-03-25  Nina Patel  Jonas Müller       No
39  2025-03-26  Nina Patel    Emma López       No

          date crew_member  contact_with symptoms
29  2025-03-24  Emma López  Alice Carter       No
35  2025-03-25  Emma López    Ben O'Hara       No
41  2025-03-26  Emma López  Jonas Müller       No


- Nina Patel had contact with Emma López **one day** before **both** became symptomatic.
- **Both** had contact with **Alice Carter 3 days before** symptom onset.
- **Both** had contact with **Jonas Müller** before symptom onset, albeit on different days.

#### Possible interpretations:
1. Possible transmission path from Emma López to Nina Patel (i.e., Emma López infected Nina Patel).
2. Alice Carter was contacted by both symptomatic individulas on the same day, suggesting that she may serve as a **central node** in the contact network.
3. Jonas Müller has separate direct contact with both prior to their symptom onset, placing him a 

In [14]:
# Tracing Alice Carter and Jonas Müller's contacts before 24th
def contact(names):
    for name in names:
        contact = contagion_log[contagion_log['crew_member'] == name]
        contact = contact[contact['date'] < '2025-03-27']
        print(contact)

In [15]:
contact2(['Alice Carter','Jonas Müller'])

          date   crew_member  contact_with symptoms
0   2025-03-20  Alice Carter    Ben O'Hara       No
6   2025-03-21  Alice Carter    Ben O'Hara       No
12  2025-03-22  Alice Carter     Leo Zhang       No
18  2025-03-23  Alice Carter           NaN       No
24  2025-03-24  Alice Carter  Jonas Müller       No
30  2025-03-25  Alice Carter    Nina Patel       No
36  2025-03-26  Alice Carter     Leo Zhang       No
          date   crew_member  contact_with symptoms
2   2025-03-20  Jonas Müller    Nina Patel       No
8   2025-03-21  Jonas Müller    Emma López       No
14  2025-03-22  Jonas Müller    Emma López       No
20  2025-03-23  Jonas Müller     Leo Zhang       No
26  2025-03-24  Jonas Müller           NaN       No
32  2025-03-25  Jonas Müller  Alice Carter       No
38  2025-03-26  Jonas Müller    Ben O'Hara       No


### Contagion analysis so far: identifying potential Patient Zero

#### Symptom onset cases
- Nina Patel and Emma López both became symptomatic on 2025-03-27, triggering a backward contact trace.
- The goal is to identify a likely index case (Patient Zero) — the earliest individual who may have seeded the infection.

#### Contact patterns observed
Both Nina and Emma had contact with **Jonas Müller and Alice Carter between 2025-03-24 and 2025-03-26**.

Specifically:
- Nina contacted Jonas on 2025-03-25
- Emma contacted Jonas on 2025-03-26
- Both contacted Alice Carter on 2025-03-24

#### Contact logs for Jonas Müller:
- Jonas contacted Nina (as early as 2025-03-20)Emma (2025-03-21 & 22)
- Jonas was contacted by Alice Carter on 2025-03-24
- Jonas has not reported any symptoms to date

#### Contact logs for Alice Carter:
- Alice was in contact with Ben, Jonas, Leo, and Nina between 2025-03-20 and 2025-03-26
- She was also contacted by both symptomatic individuals on 2025-03-24
- Alice is also not symptomatic, but frequently present in multiple contact chains

### Interpretation
1. Jonas Müller is the most likely common point of exposure:
    - He had earlier contact with both Nina and Emma than anyone else.
    - His contact pattern with both symptomatic individuals falls within plausible incubation periods.
    - His own interaction history suggests he may have contracted the infection early and remained asymptomatic.
      

2. Alice Carter is a possible amplifier or intermediary, given her frequent presence in contact chains, especially on 2025-03-24 — the only shared contact day between Nina and Emma.

### Preliminary conclusion
- **Jonas Müller is the most probable Patient Zero**, having made contact with both symptomatic crew members within a consistent exposure timeline and without showing symptoms himself.
- **Alice Carter may have acted as a secondary conduit of transmission**, warranting further monitoring.



In [16]:
from datetime import datetime
from datetime import timedelta

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [17]:
date_new = []
for d in contagion_log['date']:
    date_obj = datetime.strptime(d, "%Y-%m-%d").date()
    date_new.append(date_obj)
contagion_log['date_new'] = date_new
contagion_log.head()

,date,crew_member,contact_with,symptoms,date_new
0,2025-03-20,Alice Carter,Ben O'Hara,No,2025-03-20
1,2025-03-20,Ben O'Hara,Alice Carter,No,2025-03-20
2,2025-03-20,Jonas Müller,Nina Patel,No,2025-03-20
3,2025-03-20,Nina Patel,Leo Zhang,No,2025-03-20
4,2025-03-20,Leo Zhang,Nina Patel,No,2025-03-20


In [18]:
def contact_chain(names):
    for name in names:
        symptomatic = contagion_log[(contagion_log['crew_member'] == name) & (contagion_log['symptoms'] == 'Yes')]
        date_symp = symptomatic['date_new'].values[0]
        recent_contact = contagion_log[(contagion_log['date_new'] >= date_symp -  timedelta(days=3)) & (contagion_log['date_new'] < date_symp) & (contagion_log['contact_with']==name)]
        print(recent_contact)

In [19]:
contact_chain(['Nina Patel','Emma López'])

          date   crew_member contact_with symptoms    date_new
30  2025-03-25  Alice Carter   Nina Patel       No  2025-03-25
31  2025-03-25    Ben O'Hara   Nina Patel       No  2025-03-25
37  2025-03-26    Ben O'Hara   Nina Patel       No  2025-03-26
          date crew_member contact_with symptoms    date_new
39  2025-03-26  Nina Patel   Emma López       No  2025-03-26
